In [ ]:
!pip install skfeature-chappers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.utils import resample
from skfeature.function.similarity_based import fisher_score
import numpy as np
import pandas as pd

In [ ]:
import random
import math,time,sys, os
from matplotlib import pyplot
from datetime import datetime

In [ ]:
## -----------------Reading dataset files------------------------
train_data=pd.read_csv('/kaggle/input/labels/Combined_inter/Combined_inter/train.csv', index_col=False, header = None)
y_train = train_data[train_data.columns[-1]]
(noTrainRows, _) = train_data.shape

#Converting dataseries into pandas dataframe
y_train = y_train.to_frame()
dataframe_train = train_data.drop(train_data.columns[[-1]], axis=1)

test_data=pd.read_csv('/kaggle/input/labels/Combined_inter/Combined_inter/test.csv', index_col=False, header = None)
y_test = test_data[test_data.columns[-1]]

#Converting dataseries in pandas dataframe
y_test = y_test.to_frame()
dataframe_test = test_data.drop(test_data.columns[[-1]], axis=1)
print('Train and Test datasize : ', dataframe_train.shape, dataframe_test.shape)

In [ ]:
coln=dataframe_train.columns
coln1=y_train.columns

In [ ]:
dataframe=dataframe_train.append(dataframe_test,ignore_index=True)
y=y_train.append(y_test,ignore_index=True)
print('Combined size : ',dataframe.shape)

In [ ]:
##-----------------Removing columns contains all '0' -------------------
cols=[]
for i in range(len(dataframe.columns)):
    col1=dataframe[dataframe.columns[i]].to_numpy()
    if(np.sum(col1)==0):
        cols.append(dataframe.columns[i])

dataframe = dataframe.drop(columns=cols)
print('Size after removing unnecessary columns :', dataframe.shape)

In [ ]:
df=dataframe.copy()
df['labels']=y
print('After adding labels : ', df.shape)
df = df.replace(np.nan, 0)
df.head()

In [ ]:
x=df.iloc[:,:-1]
x.head()

In [ ]:
ln=df.shape[1]
str=df.columns[ln-1]
print(ln, str)

In [ ]:
print(x.shape)
dataset=x.to_numpy()

In [ ]:
def sort_agents(agents, obj,train_X,val_X,train_Y,val_Y):
    # sort the agents according to fitness
    #train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    (obj_function, weight_acc) = obj

    # if there is only one agent
    if len(agents.shape) == 1:
        num_agents = 1
        fitness = obj_function(agents, train_X, val_X, train_Y, val_Y, weight_acc)
        return agents, fitness

    # for multiple agents
    else:
        num_agents = agents.shape[0]
        fitness = np.zeros(num_agents)
        for id, agent in enumerate(agents):
            fitness[id] = obj_function(agent, train_X, val_X, train_Y, val_Y, weight_acc)
            
        idx = np.argsort(-fitness)
        sorted_agents = agents[idx].copy()
        sorted_fitness = fitness[idx].copy()
        
    return sorted_agents, sorted_fitness

In [ ]:
def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.99):
    # compute a basic fitness measure
    if(weight_acc == None):
        weight_acc = 0.99
    
    weight_feat = 1 - weight_acc
    num_features = len(agent)
   
    acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
    feat = (num_features - np.sum(agent))/num_features

    fitness = weight_acc * acc + weight_feat * feat
    
    return fitness

In [15]:
from sklearn.feature_selection import f_regression

# inputs:
#    X: pandas.DataFrame, features
#    yy: pandas.Series, target variable
#    K: number of features to select


X=x
yy=df[str].to_numpy()
K=x.shape[1]
print(yy[0])
print(X.shape)

# compute F-statistics and initialize correlation matrix
F = pd.Series(f_regression(X, yy)[0], index = X.columns)
corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

# initialize list of selected features and list of excluded features
selected = []
not_selected = X.columns.to_list()
Dict={}
# repeat K times
for i in range(K):
  
    # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
    if i > 0:
        last_selected = selected[-1]
        corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
        
    # compute FCQ score for all the (currently) excluded features (this is Formula 2)
    score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
    
    # find best feature, add it to selected and remove it from not_selected
    # print(score[score.argmax()])
    # print(score.max())
    # print(score.index[score.argmax()])
    best = score.index[score.argmax()]
    selected.append(best)
    not_selected.remove(best)
    Dict[best]=score.max()

mrMr=[]
for i in range(x.shape[1]):
  mrMr.append(Dict[x.columns[i]])
#print(mrMr)  

1
(9669, 4005)


/tmp/ipykernel_31/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_31/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_31/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_31/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when p

In [16]:
for i in range(len(mrMr)):
  mrMr[i]=mrMr[i]/(max(mrMr))

In [17]:
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

In [18]:
!pip install Py-FS
def fitness_score_values(data,target):
    from Py_FS.filter import PCC as PCC
    from Py_FS.filter import MI as MI
    from sklearn.feature_selection import mutual_info_classif
    from Py_FS.filter import Relief as Relief
    from sklearn.feature_selection import chi2
    
    print(data)
    print(target)
    #print('Pcc scores')
    #PCC_solution = PCC(data, target)
    #print(PCC_solution.scores)
    print('MI scores')
    mi_scores=mutual_info_classif(data, target, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)
    print((mi_scores))
    
    print('Relief scores')
    Relief_solution = Relief(data, target)
    print((Relief_solution.scores))
    
   
    shapley=[]
    b=x.shape[1]
    print(b)
    for i in range(b):
        shapley.append((mi_scores[i]+mrMr[i]+Relief_solution.scores[i])/3)
    return shapley

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 51.9 MB/s eta 0:00:0000:0100:01


In [26]:
target=df[str].to_numpy()
print(len(target), target)

9669 [1 1 1 ... 0 0 0]


In [20]:
!pip install tabulate
!pip install ReliefF
score=fitness_score_values(x.values,target)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ReliefF: filename=ReliefF-0.1.2-py3-none-any.whl size=6319 sha256=5e4fce21784f40a52baca5b8273339397a82cfc24552e155eff5cc67cd802679
  Stored in directory: /root/.cache/pip/wheels/35/bc/0b/14a0227d4829b6b27f25de250213eb348b094d075be7604707
Successfully built ReliefF
[[0.01891764 0.33715841 0.18103912 ... 0.00618417 0.06937568 0.02094024]
 [0.01134355 0.33530885 0.14427616 ... 0.00686109 0.18043566 0.15891416]
 [0.04868682 0.25834095 0.17169254 ... 0.         0.00734931 0.00738714]
 ...
 [0.         0.04179414 0.10518043 ... 0.         0.         0.        ]
 [0.00071266 0.08072688 0.25900993 ... 0.         0.00753672 0.        ]
 [0.00523137 0.05519465 0.06984088 ... 0.         0.         0.        ]]
[1 1 1 ... 0 0 0]
MI scores
[0.23655533 0.28412516 0.10714666 ... 0.34979384 0.36368237 0.365677  ]
Relief scores
[0.00819484 0.00041704 0.         ..

In [21]:
score = np.asarray(score)
print(score.shape)

(4005,)


In [27]:
# Returns the rank array
def ranking(score):
    rank = np.copy(score)
    rank_x = np.sort(score)
    for i in range(len(rank_x)):
        for j in range(len(rank_x)):
            if rank_x[i] == score[j]:
                rank[j] = i
    return np.asarray(rank) #rank

#Takes the ranking array consisting of rank in the positions and creates the 0th iteration feature map
def random_feature(rank, org_feat):
    feat_0 = np.copy(org_feat)
    cutoff = 0.25*(len(rank))
    for i in range(len(rank)):
        if rank[i] < cutoff:
            feat_0[i] = org_feat[i]
        else:
            feat_0[i] = 0
    return feat_0 #0th iteration feature map

#Mutates by deleting selected feature and replacing by non selected ones
def randomize(rank,feat_0,org_feat):
    feat_1 = np.copy(feat_0)
    replace = 0.1*(len(rank)) #count of number of mutation
    for i in range(int(replace)): #masking random selected top feats 
        index = np.random.randint(0.5*(len(rank))) #random index for mutation
        for k in range(len(rank)):
            if rank[k] == index:
                feat_1[index] = 0
    for j in range(len(rank)-int(replace),len(rank)): #top 10% unselected elements are selected
        feat_1[j] = org_feat[j]
    return feat_1 #2nd,3rd,.... iteration feature map

def mask(feat_1):
    mask = np.copy(feat_1)
    for i in range(len(feat_1)):
        if feat_1[i]>0:
            mask[i]=1
        else:
            mask[i]=0
    return mask

In [28]:
omega=0.9
def compute_accuracy(solution, trainX, trainy, testX,testy):
    cols=np.flatnonzero(solution)
    val=1
    if np.shape(cols)[0]==0:
        return val
    clf=KNeighborsClassifier(n_neighbors=9)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    error=1-clf.score(test_data,testy)

    #in case of multi objective  []
    featureRatio = (solution.sum()/np.shape(solution)[0])
    val=omega*error+(1-omega)*featureRatio
    # print(error,featureRatio,val)
    return val 

In [29]:
(a,b)=np.shape(df)
trainX = df.values[:noTrainRows,0:b-1]
testX  = df.values[noTrainRows:,0:b-1]
trainy = target[0:noTrainRows]
testy  = target[noTrainRows:]

In [30]:
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
mask_prev=mask(feat_prev)
feat_next=randomize(rank,feat_prev,x.values[0])
mask_next=mask(feat_next)
best=feat_prev
n=1000 #no. of iterations
for i in range (1,n+1):
    print("Iteration:")
    print(i)
    print(mask_next)
    #print(np.multiply(score,mask_prev))
    #print(np.multiply(score,mask_next))
    feat=np.sum(np.multiply(score,mask_prev)) ##comparison of the scores of unmasked features
    feat1=np.sum(np.multiply(score,mask_next))
    if feat1>feat:
        best=feat_next
        feat_prev=feat_next
        print("Score:")
        print(feat1)
    else:
        print("Score:")
        print(feat)
    
    feat_next=randomize(rank,feat_prev,x.values[0])
    mask_prev=mask(feat_prev)
    mask_next=mask(feat_next)

Iteration:
1
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
2
[0. 0. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
3
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
4
[1. 1. 0. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
5
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
6
[1. 1. 0. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
7
[1. 0. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
8
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
9
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
10
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
11
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
12
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
13
[0. 0. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
14
[1. 0. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
15
[1. 1. 1. ... 1. 1. 1.]
Score:
173.62605558318853
Iteration:
16
[1. 1. 0. ... 1. 1. 1.]
Score:
173.

In [31]:
#Applying mask on the entire test set and finding accuracy
masked=mask(best)
#TESTX=masked_dataset(masked,dataset)
print(1-(compute_accuracy(best,trainX,trainy,testX,testy)))

0.9557444493068186


In [ ]:
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
feat_next=randomize(rank,feat_prev,x.values[0])
best=feat_prev
n=1000 #no. of iterations
for i in range (1,n+1):
    print("Iteration:")
    print(i)
    feat=compute_accuracy(feat_prev,trainX,trainy,testX,testy)
    feat1=compute_accuracy(feat_next,trainX,trainy,testX,testy)
    if feat1<feat:
        best=feat_next
        feat_prev=feat_next
    print("Accuracy:")
    print(1-(compute_accuracy(best,trainX,trainy,testX,testy)))
    
    feat_next=randomize(rank,feat_prev,x.values[0])

Iteration:
1
Accuracy:
0.961057111397428
Iteration:
2
Accuracy:
0.9617084138879597
Iteration:
3
Accuracy:
0.9622578549960878
Iteration:
4
Accuracy:
0.9644534210222843
Iteration:
5
Accuracy:
0.9647995273461933
Iteration:
6
Accuracy:
0.965138456991405
Iteration:
7
Accuracy:
0.9652787438268248
Iteration:
8
Accuracy:
0.9655038475776894
Iteration:
9
Accuracy:
0.9673900671225172
Iteration:
10
Accuracy:
0.969246408501782
Iteration:
11
Accuracy:
0.969304354126799
Iteration:
12
Accuracy:
0.9729052489658782
Iteration:
13
Accuracy:
0.9729052489658782
Iteration:
14
Accuracy:
0.9729052489658782
Iteration:
15
Accuracy:
0.9747338799456862
Iteration:
16
Accuracy:
0.9747338799456862
Iteration:
17
Accuracy:
0.9747338799456862
Iteration:
18
Accuracy:
0.9747338799456862
Iteration:
19
Accuracy:
0.9747561941068495
Iteration:
20
Accuracy:
0.9747561941068495
Iteration:
21
Accuracy:
0.9747561941068495
Iteration:
22
Accuracy:
0.9748784380647739
Iteration:
23
Accuracy:
0.9749902630231323
Iteration:
24
Accuracy:


In [25]:
#For 25%-1000 top features
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
print(1-(compute_accuracy(feat_prev,trainX,trainy,testX,testy)))

0.9556581252484507


In [27]:
def random_feature(rank, org_feat):
    feat_0 = np.copy(org_feat)
    cutoff = 0.75*(len(rank))
    for i in range(len(rank)):
        if rank[i] < cutoff:
            feat_0[i] = org_feat[i]
        else:
            feat_0[i] = 0
    return feat_0 #0th iteration feature map

#For 75%-3000 top features
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
print(1-(compute_accuracy(feat_prev,trainX,trainy,testX,testy)))

0.9564181667603079


In [28]:
def random_feature(rank, org_feat):
    feat_0 = np.copy(org_feat)
    cutoff = 0.05*(len(rank))
    for i in range(len(rank)):
        if rank[i] < cutoff:
            feat_0[i] = org_feat[i]
        else:
            feat_0[i] = 0
    return feat_0 #0th iteration feature map

#For 5%-200 top features
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
print(1-(compute_accuracy(feat_prev,trainX,trainy,testX,testy)))

0.9404843003050412


In [31]:
def random_feature(rank, org_feat):
    feat_0 = np.copy(org_feat)
    cutoff = 0.9*(len(rank))
    for i in range(len(rank)):
        if rank[i] < cutoff:
            feat_0[i] = org_feat[i]
        else:
            feat_0[i] = 0
    return feat_0 #0th iteration feature map

#For 90%-3600 top features
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
print(1-(compute_accuracy(feat_prev,trainX,trainy,testX,testy)))

0.9537661306152588


In [32]:
def random_feature(rank, org_feat):
    feat_0 = np.copy(org_feat)
    cutoff = 0.5*(len(rank))
    for i in range(len(rank)):
        if rank[i] < cutoff:
            feat_0[i] = org_feat[i]
        else:
            feat_0[i] = 0
    return feat_0 #0th iteration feature map

#For 50%-2000 top features
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
print(1-(compute_accuracy(feat_prev,trainX,trainy,testX,testy)))

0.956980684975552


In [32]:
#For 100%-4005 top features
print(1-(compute_accuracy(x.values[0],trainX,trainy,testX,testy)))

0.960060536275795
